<a href="https://colab.research.google.com/github/1peng2333/Generalized-Interactive-PMX2Source-Pipeline-Program/blob/main/Generalized_Interactive_PMX2Source_Pipeline_Program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://docs.blender.org/manual/en/2.93/_images/editors_3dview_startup-scene_labels.png" height="250" align="right" style="height:250px">

#**Generalized Interactive PMX2Source Pipeline Program (GIPPP)**

**Development still in progress and not ready for release** (V20250317)

**This notebook is still just an interface (You could find nothing here, but could still see the general porting workflow)**

This project contains 5922 lines of Python Codes. Putting all of them in this notebook will be a nightmare to debug. Development started from 2025/Mar/9.

PMX format is an industry standard 3D model coding format used by most anime-style model, operated under Freeware License as a community supported project for MikuMikuDance [Official Website](https://sites.google.com/view/vpvp/). ([1] Higuchi et al. 2008)

This notebook is developed by SheepyLord to perform a generalized model porting that automate import, scaling, renaming, combining bones, texture conversion etc...

This notebook interacts with Blender. ([2] Blender Dev Team. 2024)

See [Proportion trick](https://github.com/sksh70/proportion_trick_script) for proportion trick script implemented as part of this notebook.




In [ ]:
#@title Setup Everything. **Do Not Interrupt When Running This Cell**. (~5 min)
import os, time, sys
!pwd

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

if not os.path.isdir("TensorFlow"):
  print("installing TensorFlow")
  os.system("pip install tensorflow[and-cuda]")
  os.system("mkdir TensorFlow")

#@markdown **To myself**:
#@markdown - docker start -i MMD2MDL
#@markdown - http://localhost:8080/?token=14a3a6afc0f343a842e79212206865f1acf96fd6771d0c94
#@markdown - Replace token.

if not os.path.isdir("scripts"):
  os.system("mkdir scripts")
  os.system("mkdir models and weights")
  os.system("mkdir 1_PMX")
  os.system("mkdir 2_Blender")
  os.system("mkdir 3_Flexes")
  os.system("mkdir 4_Blender_Scaled_Tex")
  os.system("mkdir 5_PT_export")
  os.system("mkdir 6_Complie_trials")

import numpy as np
import matplotlib.pyplot as plt
import torch
from tqdm.notebook import tqdm
import tensorflow as tf
from tensorflow import keras
import json
import pandas as pd

print("Tensorflow:", tf.version.VERSION)


/content
Tensorflow: 2.18.0


In [ ]:
#@title Model Import **Settings**
#@markdown **Basic** settings
#@markdown ---

project_name = "Megumi" #@param {type:"string"}
project_name = project_name.lower()
os.system(f"mkdir {project_name}")

mmd_file_name = "megumi_refrence" #@param {type:"string"}
#@markdown - Will search ".pmx" in 1_PMX folder.

target_model_height = "1.52" #@param {type:"string"}
#@markdown - All units in meters and kilograms.

fix_model_interations = 2 #@param ["0", "1", "2", "4"]

model_height_refrence = "head_top" #@param ["eyes", "head_top", "highest_pt_on_model"]
#@markdown - `eyes` assumes eyes is at 0.1 m below the model top.
#@markdown - `head_top` uses a pretrained 3D CNN model to identify head, and take the top position of the head box.
#@markdown - `highest_pt_on_model` have no consideration.

opts = [f"--project_name = {project_name}",
        f"--mmd_file_path = 1_PMX/",
        f"--mmd_file_name = {mmd_file_name}",
        f"--target_model_height = {target_model_height}",
        f"--fix_model_interations = {fix_model_interations}",
        f"--model_height_refrence = {model_height_refrence}",
        ]

opts = ' '.join(opts)
!python scripts/blender/1_import_and_scale.py {opts}

S1: Model Imported
S1: Refrence Imported
S1: 3D CNN Model Exectued
S1: Scaled Model to Desired Height
S1: Applied All Transforms
S1: Refrence Removed
S1: Moved Lowest Point of Model 0.05m About the Ground
S1: Blend file Written
S1: Script Completed Without Error


In [ ]:
#@title Model **Translation** into English Valve

flex_name_standard = "8siandude" #@param ["8siandude", "Valve", "RStar", "Captain"]
bone_name_standard = "8siandude" #@param ["8siandude", "Valve", "RStar", "Captain"]
use_custom_taunt = False #@param {type:"boolean"}

remove_excess_objects = True #@param {type:"boolean"}
combine_excess_bones = True #@param {type:"boolean"}

missing_translation_standard = "DeepL" #@param ["local_dict", "Google", "DeepL", "Baidu"]
#@markdown - Auto set API key needs to be implemented.

opts = [f"--flex_name_standard = {flex_name_standard}",
        f"--bone_name_standard = {bone_name_standard}",
        f"--blend_file_path = /2_Blender/",
        f"--use_custom_taunt = {use_custom_taunt}",
        f"--remove_excess_objects = {remove_excess_objects}",
        f"--combine_excess_bones = {combine_excess_bones}",
        f"--missing_translation_standard = {missing_translation_standard}",
        ]

opts = ' '.join(opts)
!python scripts/blender/2_translate.py {opts}

S2: Blend file Imported
S2: Translated using cat_mmd plugin
S2: Found 0 bad translation in Bone
S2: Found 0 bad translation in Texture
S2: Found 1 bad translation in Shapekey
S2: Removed All Special Chars
S2: Translation: Mouth ~~ => mouth_w
S2: Flex List Written
S2: Bone Renamed to Source
S2: Flexes Renamed to Source
S2: Blend File Written
S2: Script Completed Without Error


In [ ]:
#@title Model **Add and Combine** Bones

#@markdown Only optimized for toes and skirt

add_spine_bones = True #@param {type:"boolean"}
combine_splited_toes = True #@param {type:"boolean"}
combine_skirt = True #@param {type:"boolean"}
combine_skirt_scheme = "auto_odd" #@param ["auto_even", "auto_odd"]
#@markdown - Combine 6 bones to 1, usually end with 6 and 7 bones.
#@markdown - Try to switch to another one if it fails.
#@markdown - Odd for situation which there is bone at exact location of front skirt center line.
#@markdown - Even for situation which there is no bone at front skirt center line.

opts = [f"--add_spine_bones = {add_spine_bones}",
        f"--combine_splited_toes = {combine_splited_toes}",
        f"--combine_skirt = {combine_skirt}",
        f"--combine_skirt_scheme = {combine_skirt_scheme}"
        f"--blend_file_path = /2_Blender/",
]

!python scripts/blender/3_ac_bones.py {opts}

S3: Blend file Imported
Missing: ValveBiped.Bip01_Pelvis
Mislocated: ValveBiped.Bip01_Spine
Missing: ValveBiped.Bip01_Spine2
Mislocated: ValveBiped.Bip01_Spine4
Moved: ValveBiped.Bip01_Spine4
Added: Base.001
Added: ValveBiped.Bip01_Spine1.001
Renaming: Base => ValveBiped.Bip01_Pelvis
Reparent: ValveBiped.Bip01_Spine => ValveBiped.Bip01_Pelvis
Reparent: ValveBiped.Bip01_L_Thigh => ValveBiped.Bip01_Pelvis
Reparent: ValveBiped.Bip01_R_Thigh => ValveBiped.Bip01_Pelvis
Renaming: ValveBiped.Bip01_Spine1.001 => ValveBiped.Bip01_Spine2
Connecing: ValveBiped.Bip01_Spine2 => ValveBiped.Bip01_Spine4
Found 5 Correct Spine Bones
Passed
S3: Combined toes
S3: Skirt bones combined successfully
S3: Blend File Written
S3: Script Completed Without Error


In [ ]:
#@title Model **Combine** Textures and Remove Unwanted Verts

combine_textures = True #@param {type:"boolean"}
keep = "Head, Face, Expressions, Bra, Pants, Skirt, Shirt, Coat, Jacket, Socks, Shoes" #@param {type:"string"}
keep_search_mode = "arbitrary" #@param ["exact_match", "arbitrary"]

opts = [f"--combine_textures = {combine_textures}",
        f"--keep = {keep}",
        f"--keep_search_mode = {keep_search_mode}",
        f"--blend_file_path = /2_Blender/",]

!python scripts/blender/4_cb_tex.py {opts}

S4: Blend file Imported
Textures with Similar Names Were Combined
Textures count 36 => 16
S4: Removed Unwanted Verts
S4: Removed Unwanted Textures
S4: Set All Alpha to 1.0
S4: Blend File Written
S4: Script Completed Without Error


In [ ]:
#@title Apply Necessary ShapeKeys to Edit Mode

apply_shape_keys = True #@param {type:"boolean"}
print("Have to be done manually")
print("Download 4.blend and apply required shape keys.")
print("Upload process file as 5.blend and Continue")

if apply_shape_keys:
  raise FileNotFoundError

Have to be done manually
Download 4.blend and apply required shape keys.
Upload process file as 5.blend and Continue


FileNotFoundError: 

In [ ]:
#@title Split and Export

split_by = "texture" #@param ["texture", "loose_group"]
verts_limit = 12800
over_limit_split_by = "center_of_mass" #@param ["axis", "center_of_mass", "random", "loose_group"]
over_limit_rename_by = "DeepSeek" #@param ["DeepSeek", "ChatGPT"]
prompt = "Body has 2 parts, 'FrontBody' and 'BackBody', like {AAA} have 2 parts, {BBB} and {CCC}. \n Please keep the same format and fill the Bs and Cs" #@param {type:"string"}

opts = [f"--split_by = {split_by}",
        f"--verts_limit = {verts_limit}",
        f"--over_limit_split_by = {over_limit_split_by}",
        f"--blend_file_path = /2_Blender/",]

!python scripts/blender/6_sp_exp.py {opts}

S6: Blend file Imported
S6: Splited by Texture, Got 15 Body Groups
S6: Removed ShapeKeys for Shoes
S6: Auto Combined Face, Got 15 => 12 Body Groups
S6: Body Group Hair Has 19925 Verts!!!
S6: Splited Hair, by COM, Got 8956 and 10969.
S6: Waiting for DeepSeek API to respond.
S6: Renamed as 'FrontHair' and 'BackHair'
S6: Final Check Reported No Error
S6: Exported
S6: Last Blend File Written
S6: Script Completed Without Error


In [ ]:
#@title Do Textures VTFEDIT

do_textures = True #@param {type:"boolean"}
#@markdown - Just Run this, this is well-coded already.
opts = [f"project_name = {project_name}",]

!python scripts/blender/7_dt.py {opts}

S7: Found 12 Available Texture Files
S7: Copied 3 Files to Match All Material Instances
S7: Texture Renaming...
S7: Converting to VTF/VMT (~ 90s)
S7: Adjusting culls and paths
S7: Script Completed Without Error


In [ ]:
#@title Proportion Tricks

!python scripts/proportion_tricks/SPT.py {opts}

SPT: Blender is up, v2.7.3
SPT: Loaded 11 Available SMD Files
SPT: Named to gg
SPT: Ran PT1
SPT: Applied as Rest Pose
SPT: Cleared Pose Constraint
SPT: Calculating Toe to Z axis Angle
SPT: Adjusting Toe and Ankle
SPT: Layer 1 Activated
SPT: Ran PT2
SPT: Linked Modifiers
SPT: Exporting
SPT: Script Completed Without Error


In [ ]:
#@title Do Phys Model

phys_model_template = "8siandude" #@param ["8siandude", "auto_center_of_mass_by_bones", "template"]
auto_scaling = True #@param {type:"boolean"}
allow_move_bone_to_match_model = True #@param {type:"boolean"}
allow_move_phys_model_to_match_bone = False #@param {type:"boolean"}

opts = [f"phys_model_template = {phys_model_template}",
        f"project_name = {project_name}",
        f"auto_scaling = {auto_scaling}",
        f"allow_move_bone_to_match_model = {allow_move_bone_to_match_model}",
        f"allow_move_phys_model_to_match_bone = {allow_move_phys_model_to_match_bone}",
        f"--blend_file_path = /2_Blender/",]

!python scripts/blender/8_phys.py {opts}

S8: Loaded Body.smd
S8: Using Odd Version of Phys Model
S8: Loaded Refrence Phys Model
S8: Rescaling
S8: Checking Weights and Names
S8: Exorting
S8: Script Completed Without Error


In [ ]:
#@title Auto Qc and Compliing

use_complier = "Advanced_Gmod" #@param ["Gmod", "Advanced_Gmod", "SFM", "L4D2", "CSGOLegacy"]
do_defbone = True #@param {type:"boolean"}
do_hbox = True #@param {type:"boolean"}
do_jbones = True #@param {type:"boolean"}
do_phys_model = True #@param {type:"boolean"}
verbose = False #@param {type:"boolean"}

opts = [f"use_complier = {use_complier}",
        f"do_defbone = {do_defbone}",
        f"do_hbox = {do_hbox}",
        f"do_jbones = {do_jbones}",
        f"project_name = {project_name}",
        f"do_phys_model = {do_phys_model}",
        f"quiet = {verbose}",]

!python scripts/blender/9_comp.py {opts}

S9: Generated flex controllers from Face.vta
S9: Define Bone Completed
S9: Define HitBox Completed
S9: Assigned Jbones to *hair, *skirt and *breast
S9: Constructed Phys Model Settings
S9: Compling from model.qc
S9: Script Completed Without Error
S9: Model Compiled with Great Success!!!


In [ ]:
#@title Compose Folder and Zip
from google.colab import files

!python scripts/blender/10_publish.py {opts}
files.download(f'{project_name}.zip')

S10: Moved Model
S10: Moved Materials
S10: Generated Placeholder SpawnIcons
S10: Generated LUAs
S10: Moved Placeholder c_arms
S10: Clean_up
S10: Zipping Folder


# Citations
(Not finished yet)

[1] Higuchi et al. (2008). MikuMikuDance (Version 9.32) [Computer software]. 	https://sites.google.com/view/vpvp

[2] Blender Development Team. (2022). Blender (Version 3.1.0) [Computer software]. https://www.blender.org